In [1]:
import numpy as np
import pandas as pd

In [31]:
def read_command(path):
    with open(path) as f:
        lines = f.readlines()
        
    com = []    
    count = 0
    for l in lines:
        if count == 0 and '# Example' in l:
            count += 1
        elif count == 1 and l == '#\n':
            count += 1
        elif count == 2 and '########################' in l:
            count += 1
        elif count ==3 and l == '\n':
            count += 1
        elif count == 4:
            while ',' in com:
                l = l.replace(',','')
            com.append(l.strip())
            count = 0
            
    return com

path = './EGPSR Examples.txt'
com_lst = read_command(path)  
print('len: ',len(com_lst))
com_lst[:10]

len:  10000


['Robot please describe -B_say- the objects in the sink to me -I_say-',
 'Find -B_find- the basket -B_find_obj- in the corridor -B_find_dest-',
 'Locate -B_find- Skyler -B_find_per-, pinpoint -B_find- Francis -B_find_per- at the end -i- table -B_find_dest-, and accompany -B_guide- her -B_guide_per- to the exit -B_guide_dest- please',
 'Pick -B_take- up -I_take- the orange -B_take_obj- from the dining -i- table -B_take_sour-, pick -B_take- up -I_take- the fruits -B_take_obj- from the side -i- table -B_take_sour-, and spot -B_find- Linda -B_find_per- at the dishwasher -B_find_dest- please',
 'Robot please tell -B_say- me how many children -i- are -i- pointing -i- to -i- the -i- left in the kitchen -I_say-',
 'Find -B_find- James -B_find_per- in the dining -i- room -B_find_dest-, accompany -B_follow- him -B_follow_per-, leave -B_take- the apple -B_take_obj- in the living -i- room -B_take_dest- please',
 'Tell -B_say- me how many women -i- are -i- pointing -i- to -i- the -i- left in the -i

In [32]:
import itertools

def improve_labels(com_lst):
    res = []
    for i,com in enumerate(com_lst):
        com = com.split()
        if '-B_say-' not in com and '-i-' not in com:
            res.append(com)
            continue
        elif '-B_say-' in com:
            if '-I_say-' not in com:
                labelled = com
            else:
                b_idx = com.index('-B_say-') + 1
                e_idx = com.index('-I_say-') -1
                first = com[:b_idx]
                end = com[e_idx:]
                
                need_label = com[b_idx:e_idx]
                
                #rem all -i- in the 'say' cases
                need_label = [ele for ele in need_label if ele != '-i-']
                
                interleave = list(itertools.chain(*zip(need_label, ['-I_say-']*len(need_label))))
                labelled = first + interleave + end
                
                com = labelled.copy()
                
        elif '-i-' in com: 
            string = ' '.join(com)
            while '-i- -i-' in string:
                string = string.replace('-i- -i-', '-i-')
                
            while '-i- -' in string:
               # print(string)
                string = string.replace('-i- -','-')
                #print('#',string)
                
            com = string.split()
            
                
                
            idx_lst = []
            for idx,ele in enumerate(com):
                if ele == '-i-':
                    if len(idx_lst) == 0: 
                        idx_lst.append([idx])
                    elif idx == idx_lst[-1][-1] + 2:
                        idx_lst[-1].append(idx)
                    else:
                        idx_lst.append([idx])
            if i == 1:
                print(com)
                print(idx_lst)
            for lst in idx_lst:
                end_label = com[lst[-1]+2]
                com[lst[0]] = end_label
                if 'B' in end_label:
                    com[lst[-1]+2] = com[lst[-1]+2].replace('B','I')
                    
                for i,idx in enumerate(lst):
                    if i == 0:
                        continue
                    else:
                        label = com[lst[-1]+2]
                        com[idx] = label
                        
            labelled = com.copy()
               
                
        res.append(labelled)     
    return res

res = improve_labels(com_lst)

In [33]:
#res[3]
res[12]

['Give',
 '-B_take-',
 'me',
 '-B_take_dest-',
 'the',
 'right-most',
 '-B_take_obj-',
 'cutlery',
 '-I_take_obj-',
 'from',
 'the',
 'sink',
 '-B_take_sour-',
 'please']

# Generate Seq Out ( Copied from mbot, but changed intro to emplty lst)

In [34]:
def generate_seq_out(res):
    seq_in = []
    seq_out = []
    for i,com in enumerate(res):
        in_lst = []
        out_lst = []
        for ele in com:
            if ele[0] != '-':
                in_lst.append(ele)
                out_lst.append('O')
            else:
                if i == 76:
                    print(out_lst)
                while '-' in ele:
                    ele = ele.replace('-','')
                    
                out_lst[-1] = ele
                if ',' in ele:
                    out_lst[-1] = ele.replace(',','')

        seq_in.append(in_lst)
        seq_out.append(out_lst)
    return seq_in, seq_out

seq_in, seq_out = generate_seq_out(res)

['O']
['B_take', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'B_take', 'O', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'B_take', 'O', 'B_take_obj', 'O', 'O', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'B_take', 'O', 'B_take_obj', 'O', 'O', 'B_take_sour', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'B_take', 'O', 'B_take_obj', 'O', 'O', 'B_take_sour', 'I_take_sour', 'O', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'B_take', 'O', 'B_take_obj', 'O', 'O', 'B_take_sour', 'I_take_sour', 'O', 'B_take', 'O', 'O']
['B_take', 'B_take_obj', 'O', 'O', 'B_take_dest', 'I_take_dest', 'B_take', 'O', 'B_take_obj', 'O', 'O', 'B_take_sour', 'I_take_sour', 'O', 'B_take', 'O', 'B_take_obj', 'O', 'O', 'O']


In [25]:
seq_out[12]

['B_take',
 'B_take_dest',
 'O',
 'B_take_obj',
 'I_take_obj',
 'O',
 'O',
 'B_take_sour',
 'O']

In [26]:
seq_out[76]

['B_take',
 'B_take_obj',
 'O',
 'O',
 'B_take_dest',
 'I_take_dest',
 'B_take',
 'O',
 'B_take_obj',
 'O',
 'O',
 'B_take_sour',
 'I_take_sour',
 'O',
 'B_take',
 'O',
 'B_take_obj',
 'O',
 'O',
 'B_take_sour',
 'O']

In [23]:
# def generate_seq_out(tasks):
#     c = 0
#     sentences = []
#     outputs = []
#     for v in range(len(tasks)):

# #         try:
# #             task = tasks[v].split(' ')
# #         except:
# #             print('error at {}, check if the items in tasks are strings tasks[v] = {}'.format(v, tasks[v]))
# #             break
#         task = tasks[v]
#         # intro = intros[c].split(' ')
#         intros = []
#         intro = []

#         sentence = []
#         output = []
#         # appending intro if the total length is less than or equal to 15 (ref: Pedro master thesis) ############################################################
#         task_with_intro = []
#         if v%4 == 0 and (len(intro) + len(task)) <= 15:
#             for x in intro:
#                 task_with_intro.append(x)
#             c = c + 1

#         # reinitiating intros
#         if c == len(intros):
#             c = 0

#         # appending task
#         for x in task:
#             task_with_intro.append(x)

#         # appending the task with introduction to the sentence list
#         for h in range(len(task_with_intro)):
#             if not task_with_intro[h].startswith('-'):
#                 sentence.append(task_with_intro[h])
#                 # appending outputs according to the input
#                 if h < len(task_with_intro)-1:
#                     if task_with_intro[h+1].startswith('-'):
#                         l = task_with_intro[h+1]
#                         l = l.replace('-', '')
#                         output.append(l)
#                     else:
#                         output.append('O')
#                 else:
#                     output.append('O')
#         # split sentences and tags
#         sentences.append(sentence)
#         outputs.append(output)
#     return outputs

# seq_out =  generate_seq_out(res)

In [32]:
seq_in[12]

['Give', 'me', 'the', 'right-most', 'cutlery', 'from', 'the', 'sink', 'please']

In [33]:
seq_out[12]

['B_take',
 'B_take_dest',
 'O',
 'B_take_obj',
 'I_take_obj',
 'O',
 'O',
 'B_take_sour',
 'O']

In [19]:
com_lst[12]#.split()

'Give -B_take- me -B_take_dest- the right-most -i- cutlery -B_take_obj- from the sink -B_take_sour- please'

In [27]:
seq_in = [[string for string in com.split() if '-' not in string] for com in com_lst]
print(seq_in[12])
seq_out[12]

['Give', 'me', 'the', 'cutlery', 'from', 'the', 'sink', 'please']


['B_take',
 'B_take_dest',
 'O',
 'B_take_obj',
 'I_take_obj',
 'O',
 'O',
 'B_take_sour',
 'O']

In [33]:
seq_in

[['Please',
  'spot',
  'a',
  'orange',
  'in',
  'the',
  'living',
  'room',
  'take',
  'the',
  'paprika',
  'and',
  'tell',
  'the',
  'time'],
 ['Put',
  'the',
  'orange',
  'into',
  'the',
  'basket',
  'on',
  'the',
  'storage',
  'table',
  'please'],
 ['Could',
  'you',
  'please',
  'tell',
  'me',
  'how',
  'many',
  'men',
  'in',
  'the',
  'corridor',
  'are',
  'sitting'],
 ['Pick',
  'up',
  'a',
  'cutlery',
  'from',
  'the',
  'dining',
  'table',
  'and',
  'put',
  'it',
  'into',
  'the',
  'basket',
  'on',
  'the',
  'counter'],
 ['Grasp',
  'the',
  'fruits',
  'from',
  'the',
  'dining',
  'table',
  'put',
  'it',
  'on',
  'the',
  'counter',
  'and',
  'place',
  'paprika',
  'on',
  'the',
  'end',
  'table'],
 ['Look',
  'for',
  'Francis',
  'at',
  'the',
  'couch',
  'pinpoint',
  'Jennifer',
  'in',
  'the',
  'living',
  'room',
  'and',
  'find',
  'Mary'],
 ['Hand', 'me', 'some', 'orange', 'juice', 'in', 'a', 'bowl', 'please'],
 ['Robot',
 

In [32]:
seq_out

[['B_take',
  'I_take',
  'O',
  'B_take_obj',
  'O',
  'O',
  'B_take_sour',
  'I_take_sour',
  'O',
  'B_take',
  'B_take_obj',
  'O',
  'O',
  'B_take_dest',
  'O'],
 ['B_know',
  'I_know',
  'I_know',
  'B_find_per',
  'I_find_per',
  'I_find_per',
  'I_find_per,',
  'O',
  'B_greet',
  'B_greet_per',
  'O'],
 ['O',
  'O',
  'B_go',
  'O',
  'O',
  'B_go_dest,',
  'B_find',
  'I_find',
  'B_find_per,',
  'O',
  'B_guide',
  'B_guide_per'],
 ['B_take',
  'O',
  'B_take_dest',
  'O',
  'B_take_obj',
  'I_take_obj',
  'O',
  'O',
  'B_take_sour'],
 ['B_say',
  'I_say',
  'I_say',
  'I_say',
  'I_say',
  'I_say',
  'I_say',
  'I_say',
  'I_say',
  'I_say'],
 ['B_find',
  'O',
  'B_find_obj,',
  'B_find',
  'B_find_per',
  'I_find_per',
  'I_find_per',
  'I_find_per,',
  'I_find_per,',
  'O',
  'B_guide',
  'B_guide_per',
  'O',
  'O',
  'B_guide_dest'],
 ['O',
  'O',
  'O',
  'B_greet',
  'O',
  'B_greet_per',
  'I_greet_per',
  'I_greet_per',
  'I_greet_per',
  'I_greet_per'],
 ['B_ta

# Save seq_in and Seq_out

In [35]:
with open('dataset_seq_in.txt', 'w') as f:
    for lst in seq_in:
        com = ' '.join(lst)
        f.write(com)
        f.write('\n')
        
        
with open('dataset_seq_out.txt', 'w') as f:
    for lst in seq_out:
        com = ' '.join(lst)
        f.write(com)
        f.write('\n')

## Generate a readable file

In [36]:
max_seq_len = max(len(x) for x in seq_in)

seq_dict = {(i-1 if i != 0 else 'ID'):[' ']*(len(seq_in)*3) for i in range(max_seq_len + 1)}

for token_idx,t_lst in seq_dict.items():
    for com_idx in range(len(seq_in)):
        df_idx = com_idx*3
        
        if token_idx == 'ID':
            t_lst[df_idx] = com_idx
            t_lst[df_idx+1] = ' '
            t_lst[df_idx+2] = '------------------'
            continue
        
        com_seq_lst = seq_in[com_idx]#dataset_df.loc[com_idx,'Commands'].split()
        max_len = len(com_seq_lst)
        if token_idx > max_len-1:
            continue
        com_token = com_seq_lst[token_idx]
        out_token = seq_out[com_idx][token_idx]#dataset_df.loc[com_idx,'seq_out'][token_idx]
        t_lst[df_idx] = com_token
        t_lst[df_idx+1] = out_token
        t_lst[df_idx+2] = '------------------'
        seq_dict.update({token_idx:t_lst})
        
    
df_read = pd.DataFrame(seq_dict)
df_read.to_csv('dataset.csv', index = False)
df_read.head(14)

,ID,0,1,2,3,4,5,6,7,8,...,16,17,18,19,20,21,22,23,24,25
0,0,Robot,please,describe,the,objects,in,the,sink,to,...,,,,,,,,,,
1,,O,O,B_say,I_say,I_say,I_say,I_say,I_say,I_say,...,,,,,,,,,,
2,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,...,,,,,,,,,,
3,1,Find,the,basket,in,the,corridor,,,,...,,,,,,,,,,
4,,B_find,O,B_find_obj,O,O,B_find_dest,,,,...,,,,,,,,,,
5,------------------,------------------,------------------,------------------,------------------,------------------,------------------,,,,...,,,,,,,,,,
6,2,Locate,Skyler,pinpoint,Francis,at,the,end,table,and,...,,,,,,,,,,
7,,B_find,B_find_per,B_find,B_find_per,O,O,B_find_dest,I_find_dest,O,...,,,,,,,,,,
8,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,------------------,...,,,,,,,,,,
9,3,Pick,up,the,orange,from,the,dining,table,pick,...,and,spot,Linda,at,the,dishwasher,please,,,
